## Stappenplan en verdeling

1. aanmaken github
2. data preprocessing (verwijder duplicates and NaN)
3. bepaal de relevante features van de descriptors via een PCA
4. wat wordt test en wat wordt train
5. aanmaken model voor het maken van voorspellingen
    5.1 uitproberen welke methodes er toepasbaar zijn (random forest, k-NN)
    5.2 bepalen welke methode het beste is
6. verslag en presentatie 

1. Richard
2. Luuk, Richard
3. Robin, Tijn, Rik & Yael
4. 
5. Julien & Branco
6. 

## 5.1 Onderzoek over random forest methode
The random forest algorithm is an extension of the bagging method as it utilizes both bagging and feature randomness to create an uncorrelated forest of decision trees. While decision trees consider all the possible feature splits, random forests only select a subset of those features.

Random forest algorithms have three main hyperparameters, which need to be set before training. These include node size, the number of trees, and the number of features sampled. From there, the random forest classifier can be used to solve for regression or classification problems.

> Dit betekent dat alle descriptors onze features zijn waarvan per tree een random keuze wordt gemaakt. 
> The nodesize parameter specifies the minimum number of observations in a terminal node. Setting it lower leads to trees with a larger depth which means that more splits are performed until the terminal nodes.

The random forest algorithm is made up of a collection of decision trees, and each tree in the ensemble is comprised of a data sample drawn from a training set with replacement, called the bootstrap sample. Of that training sample, one-third of it is set aside as test data, known as the out-of-bag (oob) sample, which we’ll come back to later. Another instance of randomness is then injected through feature bagging, adding more diversity to the dataset and reducing the correlation among decision trees. Depending on the type of problem, the determination of the prediction will vary.

> Dit geeft dus aan dat onze verschillende moleculen niet allemaal tegelijk moeten worden gebruikt maar in secties. De grote van deze sectie is denk ik voor onszelf te bepalen. 

In [5]:
#Useful functions:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
#Hiermee importeren we de RandomForestClassifier functie.

# Load the dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier()
#RandomForestClassifier() kan veel parameters gebruiken. De belangrijkste hiervan is n_estimators. Deze parameter bepaalt de hoeveelheid decision trees. Sommige parameters, zoals max_leaf_nodes, welke de maximale hoeveelheid verschillende ‘kruispunten’ in elke decision tree bepaalt, kunnen worden gebruikt om het model te optimaliseren.

rf.fit(X_train, y_train)
#Deze functie fit het model op onze data. Xtrain en Ytrain zijn in dit geval training data. Dat is dus die file tested_molecules.csv. 

Ypredicted = rf.predict(X_test)
#Nadat het model is gefit, kunnen met deze functie de waardes voor onze test data berekend worden.

ChatGPT test

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import load_iris

# Load the dataset
data = load_iris()
X = data.data
y = data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)

Accuracy: 1.0
Confusion Matrix:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

